<font size=6> <b> SMU Lab Three - MSDS7331 - Machine Learning-1 </b> </font>

<font size=5> <b> Summer 2020 Group - Sachin, Ikenna, Edgar, Dustin </b></font> 




<img src="https://github.com/sachinac/ML7331/blob/master/data/CustomersWhoBought3-2.jpg?raw=true"> 

<p align="center"><font size=5> <b> Movie Recommender System  </b></font> </p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1">Import Libraries</a></span></li><li><span><a href="#Business-Understanding-1" data-toc-modified-id="Business-Understanding-1-2">Business Understanding 1</a></span></li><li><span><a href="#Data-Understanding-1" data-toc-modified-id="Data-Understanding-1-3">Data Understanding 1</a></span></li><li><span><a href="#Data-Understanding-2" data-toc-modified-id="Data-Understanding-2-4">Data Understanding 2</a></span></li><li><span><a href="#Model-and-Evaluations-1" data-toc-modified-id="Model-and-Evaluations-1-5">Model and Evaluations 1</a></span></li><li><span><a href="#Model-and-Evaluations-2" data-toc-modified-id="Model-and-Evaluations-2-6">Model and Evaluations 2</a></span></li><li><span><a href="#Model-and-Evaluations-3" data-toc-modified-id="Model-and-Evaluations-3-7">Model and Evaluations 3</a></span></li><li><span><a href="#Model-and-Evaluations-4" data-toc-modified-id="Model-and-Evaluations-4-8">Model and Evaluations 4</a></span></li><li><span><a href="#Deployment" data-toc-modified-id="Deployment-9">Deployment</a></span></li><li><span><a href="#Exceptional-Work" data-toc-modified-id="Exceptional-Work-10">Exceptional Work</a></span></li><li><span><a href="#References" data-toc-modified-id="References-11">References</a></span></li></ul></div>

# Import Libraries

# Business Understanding 1

# Data Understanding 1

# Data Understanding 2

# Model and Evaluations 1

# Model and Evaluations 2

# Model and Evaluations 3

# Model and Evaluations 4

# Deployment

# Exceptional Work

# References

1. Dataset obtained from kaggle : <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset"> click here </a>
2. Apple <a href="https://apple.github.io/turicreate/docs/userguide/recommender/"> article </a> on Recommender. 